In [2]:
%config InlineBackend.figure_format = 'retina'
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.vision import *
import pandas as pd
import bq_helper
from bq_helper import BigQueryHelper
import re
import torch
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# When That Photo?
Goal of this project is to train a ML network to provide an estimate of when a photo was taken, where the estimate is a range of years (e.g., 1890-1900)

# Setup Access Via Big Query

In [4]:
met = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="the_met")

In [5]:
bq_assistant = BigQueryHelper("bigquery-public-data", "the_met")
bq_assistant.list_tables()

['images', 'objects', 'vision_api_data']

In [19]:
bq_assistant.table_schema('objects')

,name,type,mode,description
0,object_number,STRING,NULLABLE,None
1,is_highlight,BOOLEAN,NULLABLE,None
2,is_public_domain,BOOLEAN,NULLABLE,None
3,object_id,INTEGER,NULLABLE,None
4,department,STRING,NULLABLE,None
5,object_name,STRING,NULLABLE,None
6,title,STRING,NULLABLE,None
7,culture,STRING,NULLABLE,None
8,period,STRING,NULLABLE,None
9,dynasty,STRING,NULLABLE,None


## Exploratory Queries for Photos

In [28]:
images_url_query = """SELECT 
object_id,
object_number,
title,
is_public_domain
FROM `bigquery-public-data.the_met.objects`
WHERE department = "Photographs";
        """
images_url_response = met.query_to_pandas_safe(images_url_query, max_gb_scanned=10)
images_url_response.head(10)

,object_id,object_number,title,is_public_domain
0,281964,1997.251,[Album of 67 Photographic Studies of Self-Defe...,True
1,266370,1991.1073.67,Views in India - Beejapoor,True
2,289190,36.37 (1),[Supplemento al no. 6 anno 2o del Ricoglitore ...,True
3,289200,36.37 (11),"[Ricoglitore Di Cognizioni Utili. Anno 2o, No....",True
4,289231,36.37 (43),[Notizie sopra le Immagini Fotogeniche],True
5,266313,1990.1181,[Album of 131 Views of a French Family & Their...,True
6,285704,2005.100.1094,[View of Nice],True
7,288089,2007.234,[Spreading Oak with Seated Figure],True
8,271079,66.634.31,Leaf of the Foxglove,True
9,631026,2013.1098.2,[Street with Lamp Post and Wine Shop],True


## Photographs Query
Extract some basic info about the object from the `objects` table, and `JOIN` with the `images` table to get the relevant GCloud Storage pointer. 

Export all a subset of this data to a CSV to be used in a bash script for executing `gsutil cp` to download each image

In [20]:
gcs_url_query = """
SELECT a.object_id, a.object_name, a.title, a.object_date, b.gcs_url
FROM `bigquery-public-data.the_met.objects` a
JOIN (
  SELECT object_id, gcs_url
  FROM `bigquery-public-data.the_met.images`
) b
ON a.object_id = b.object_id
WHERE a.department = "Photographs"
AND ends_with(lower(b.gcs_url), '/0.jpg')
    """
gcs_url_response = met.query_to_pandas_safe(gcs_url_query, max_gb_scanned=10)
gcs_url_response.head(10)

,object_id,object_name,title,object_date,gcs_url
0,281964,Album,[Album of 67 Photographic Studies of Self-Defe...,ca. 1895,gs://gcs-public-data--met/281964/0.jpg
1,266370,Album,Views in India - Beejapoor,1870s–80s,gs://gcs-public-data--met/266370/0.jpg
2,289190,Ephemera,[Supplemento al no. 6 anno 2o del Ricoglitore ...,1839,gs://gcs-public-data--met/289190/0.jpg
3,289200,Ephemera,"[Ricoglitore Di Cognizioni Utili. Anno 2o, No....",1839,gs://gcs-public-data--met/289200/0.jpg
4,289231,Ephemera,[Notizie sopra le Immagini Fotogeniche],1840,gs://gcs-public-data--met/289231/0.jpg
5,266313,Album,[Album of 131 Views of a French Family & Their...,1880s–1900s,gs://gcs-public-data--met/266313/0.jpg
6,285704,Photomechanical print,[View of Nice],ca. 1880,gs://gcs-public-data--met/285704/0.jpg
7,288089,Paper negative,[Spreading Oak with Seated Figure],1850s,gs://gcs-public-data--met/288089/0.jpg
8,271079,Negative; Photograph,Leaf of the Foxglove,1839,gs://gcs-public-data--met/271079/0.jpg
9,631026,Waxed paper negative,[Street with Lamp Post and Wine Shop],1850s,gs://gcs-public-data--met/631026/0.jpg


Grab the `object_id` and `gcs_url` columns to be used by the bash download script

In [ ]:
gcs_url_response.loc[:, ['object_id','gcs_url']].to_csv(r'/data/kaggle/met/images.csv', header=None, index=None, sep=',', mode='a')

The bash download script handles the fact that trying to pipe a list of URLs to download into `gsutil cp` ignores the directory structure when downloading to the destination and places everything in the same folder (i.e., if any of the files have the same filename, they will overwrite each other).

## `gsutil` Download Script

```
#!/bin/bash
while IFS=, read -r col1 col2
do
    gsutil -m cp -r $col2 /data/kaggle/met/images/$col1
done < /data/kaggle/met/images.csv
```